In [2]:
import numpy as np
import tensorflow as tf

In [2]:
class FuzzySets(object):
    """
    This class is written to define a fuzzy set for an input. Here, by fuzzy set
    we refer to a set of membership functions for an input.
    
    """
    
    def __init__(self):
        
        self.mfs  = {}
        self.rng  = [-10, 10]
    
    def add_mf(self, mf_name, mf_type, mf_params):
        """
        This function will add a membership function to the fuzzy sets. 
        
        params:
            mf_name: string, the name for membership function. You can 
                     use any names. Note that this name will be used when 
                     you want to define the rules or evaluate the membership
                     function for a given input. 
            mf_type: string, the type of the membership function can be "trimf"
                     or "trapmf" at this time. 
            mf_params: The parameters for the membership function. if you choose
                       "trimf" you need to enter 3 and if you enter "trapmf" you
                       should enter 4 values. 
        
        """
        
        for i in range(len(mf_params) - 1):
            assert mf_params[i] <= mf_params[i + 1] , 'Parameter ' +\
            str(i) + "should be less than or equal to parameter" + str(i+1)
            
            if mf_params[i]==mf_params[i + 1]:
                mf_params[i + 1] += 0.0000001

        self.mfs[mf_name] = {'type':mf_type, 'params':mf_params} 
    
    def eval_mf(self, mf_name, inp):
        """
        given the name of membership function and input value, this function
        evaluates the membership of that input to the membership function. 
        params:
            mf_name: the name of the membership function. 
            inp: a scalar or a numpy array.
            
        """
        if type(inp) in [float, int]:
            inp = np.array(inp)
        elif type(inp) == tf.python.ops.resource_variable_ops.ResourceVariable:
            inp = inp.numpy()
        elif type(inp) == np.ndarray:
            inp = inp
            
        if self.mfs[mf_name]['type'] == 'trimf':
            output = self.trimf(self.mfs[mf_name]['params'], inp)
        elif self.mfs[mf_name]['type'] == 'trapmf':
            output = self.trimf(self.mfs[mf_name]['params'], inp)
        
        return output
    
        
    def evaluate(self, inp):
        """
        By taking an input, this function will caclulate the membership of the input
        to all membership functions. 
        inp: scalar, numpy array or tensorflow variable. 
        
        """
        if type(inp) in [float, int]:
            inp = np.array(inp)
        elif type(inp) == tf.python.ops.resource_variable_ops.ResourceVariable:
            inp = inp.numpy()
        elif type(inp) == np.ndarray:
            inp = inp
        
        output = np.zeros(inp.shape)
        
        for mf in self.mfs:
            output = np.maximum(output, self.eval_mf(mf, inp))
        
        return output
        
        
        
    def trimf(self, param, inp):
        output = np.maximum(np.minimum((inp - param[0])\
                                       / (param[1] - param[0]),\
                                       (param[2] - inp)\
                                       / (param[2] - param[1]) ), 0)
        return output
    
    def trapmf(self, param, inp):
        output = np.maximum(np.minimum(np.minimum((inp - param[0])\
                                                 / (param[1]-param[0]), 1)\
                                      , (param[3]-inp)/(param[3]-param[2]), 0))
        
        return output 
    
    
    

In [9]:
class FIS:
    
    def __init__(self, inputs = {},
                 outputs = {},
                 rules = {},
                 defuzzification = 'com',
                 AndMethod = 'min',
                 OrMethod = 'max',
                 resolution = 0.0001):
        """
        params:
            inputs: A dictionary which its keys are the names for the input sets
                    and their values are FuzzySet objects. 
            outputs: A dictionary which its keys are the names for the input sets
                    and their values are FuzzySet objects. 
            rules: A dictionary which its keys are the names for the rules
                    and their values are a string. See the examples for more information. 
            defuzzification: The defuzzification method. At this time, only 'com' (center of mass) is
                             available. 
            AndMethod: It can be 'min' or 'product'
            OrMethod : It can be 'max' or 'sum'
            resolution: The calculations are discrete, this parameters shows the distance between samples.
        
        """
        
        self.inputs = inputs
        self.outputs = outputs
        self.rules = rules
        self.defuzzification = defuzzification
        self.AndMethod = AndMethod
        self.OrMethod = OrMethod
        self.resolution = resolution
    
    def add_rule(self, rule, rule_name =0):
        """
        This function adds a rule to the FIS. 
        params:
            rule: a string. See the examples for more information
            rule_name: you can specify a name to the rule, if you dont, it will choose a dafualt method to 
                       name the rules. 
        """
        if rule_name ==0:
            self.rules[len(self.rules)] = rule
        else:
            self.rules[rule_name] = rule
            
    def eval_rule(self, rule, inp):
        
        if len(self.inputs) == 2:
            rule = rule.replace('is', '').replace('then', '').replace('if', '')
            rule = rule.split()
            mu_1 = self.inputs[rule[0]].eval_mf(rule[1], inp[rule[0]])
            mu_2 = self.inputs[rule[3]].eval_mf(rule[4], inp[rule[3]])

            if rule[2].lower() == 'and':

                if self.AndMethod == 'min':
                    mu = np.minimum(mu_1, mu_2)

                elif self.AndMethod == 'product':
                    mu = mu_1 * mu_2

            elif rule[2].lower() == 'or':

                if self.OrMethod == 'max':
                    mu = np.maximum(mu_1, mu_2)

                elif self.OrMethod == 'sum':
                    mu = np.minimum(mu_1 + mu_2, 1)
        
            out_rng = self.outputs[rule[5]].rng
            out_interval = np.arange(out_rng[0], out_rng[1], self.resolution)
            output = self.outputs[rule[5]].eval_mf(rule[6], out_interval)
            output = np.repeat(output.reshape(-1, 1), mu.shape[0], axis = 1)
            output = np.minimum(output, mu)
            out_name = rule[5]
                    
        elif len(self.inputs) == 1:
            
            rule = rule.replace('is', '').replace('then', '').replace('if', '')
            rule = rule.split()
            mu = self.inputs[rule[0]].eval_mf(rule[1], inp[rule[0]])            
        
            out_rng = self.outputs[rule[2]].rng
            out_interval = np.arange(out_rng[0], out_rng[1], self.resolution)
            output = self.outputs[rule[2]].eval_mf(rule[3], out_interval)
            output = np.repeat(output.reshape(-1, 1), mu.shape[0], axis = 1)
            output = np.minimum(output, mu)
            out_name = rule[2]
        
        return output, out_name
    
    def evaluate(self, inp):
        
        flag = 0
        
        output = {out:0 for out in self.outputs}
        
        for rule in self.rules:
            
            if flag == 0:
                temp_output, temp_name = self.eval_rule(self.rules[rule], inp)
                output[temp_name] = temp_output
                flag =1
            else:
                temp_output, temp_name = self.eval_rule(self.rules[rule], inp)
                output[temp_name] = np.maximum(temp_output, output[temp_name])
                
        return output
                
    
    def defuzzyfy(self, output):
        """
        This function will defuzzyfy the outputs
        
        """
        
        defuzzed = {}
        
        if self.defuzzification == 'com':
            
            for out in output:
                rng = self.outputs[out].rng
                x_interval = np.arange(rng[0], rng[1], self.resolution)
                
                y_temp = np.linspace(0, 1, 50).reshape(-1, 1)
                y_temp = np.repeat(y_temp, x_interval.shape[0], axis = 1)
                y_temp = np.int32(y_temp <= output[out].T)
                x_com  = np.sum(y_temp * x_interval * 1/50.0 * self.resolution)
                area = np.sum(np.array(output[out].tolist()[0:-1])*self.resolution)
                defuzzed[out] = 0 if area==0 else x_com/area
        
        return defuzzed
                
    def run(self, inputs):
        """
        Given an input, this function will calculate the output based on the sepcified rules. 
        
        """
        for inp in inputs:
            if type(inputs[inp]) != np.ndarray:
                inputs[inp] = np.array([inputs[inp]])
            
        
        out_1 = self.evaluate(inputs)
        out_2 = self.defuzzyfy(out_1)
        
        return out_2
        

 ## Examples

### Example 1:

Here we want to define a fuzzy system with 2 inputs and a single output. 

In [247]:
# first you should use the FuzzySets to define the first input:
inp_set1 = FuzzySets()

# then you can add the membership functions to this input set. 


# The first membership function name is "NB". This function's type is
#"trimf". As you know, this type of membership function takes 3 parameters 
# which are the coordinates of the vertices of the triangle. 
inp_set1.add_mf('NB', 'trimf', [-2, -1, 0]) 


# The second membership function name is "Z". This function's type is
#"trimf".
inp_set1.add_mf('Z', 'trimf', [-1, 0, 1])
# The third membership function name is "Z". This function's type is
#"trimf".
inp_set1.add_mf('PB', 'trimf', [0, 1, 2])


# Defining th first input set is finished. Then we will add another inout to our system:

inp_set2 = FuzzySets()

inp_set2.add_mf('NB', 'trimf', [-1, -0.5, 0])
inp_set2.add_mf('Z', 'trimf', [-0.5, 0, 0.5])
inp_set2.add_mf('PB', 'trimf', [0, 0.5, 1])


# After defining the inputs, we will define the outputs. Note that for output sets, you need to determine the 
# range of the output.

out_set1 = FuzzySets()
out_set1.rng = [0, 1] # By this line, we will tell our system that the range of the output for this set is between
                      # 0 and 1

# defining the outptut membership function is the same as before. 

out_set1.add_mf('Z', 'trimf', [0, 0.3, 0.6])
out_set1.add_mf('PS', 'trimf', [0.3, 0.6, 0.9])
out_set1.add_mf('PB', 'trimf', [0.6, 0.8, 1])


# After defining the input sets and the output sets, we should define the Fuzzy System:
fis = FIS(inputs={'inp1':inp_set1, 'inp2':inp_set2}, outputs = {'out1':out_set1})


# After defining the Fuzzy system, we should define the rules:

fis.add_rule('if inp1 is NB and inp2 is NB then out1 is Z')
fis.add_rule('if inp1 is Z and inp2 is Z then out1 is PS')
fis.add_rule('if inp1 is PB and inp2 is PB then out1 is PB')

In [248]:
# And here is how we can estimate the output by feeding the inputs to the FIS:
inputs = {'inp1': 1, 'inp2':0.5}
y = fis.run(inputs)
print(y)

{'out1': 0.8180032045008013}


In [226]:
# We can see the specifications for the first input set:
inp_set1.mfs

{'NB': {'type': 'trimf', 'params': [-2, -1, 0]},
 'Z': {'type': 'trimf', 'params': [-1, 0, 1]},
 'PB': {'type': 'trimf', 'params': [0, 1, 2]}}

In [227]:
# for the second input set:
inp_set2.mfs

{'NB': {'type': 'trimf', 'params': [-1, -0.5, 0]},
 'Z': {'type': 'trimf', 'params': [-0.5, 0, 0.5]},
 'PB': {'type': 'trimf', 'params': [0, 0.5, 1]}}

In [228]:
# and for the output sets:
out_set1.mfs

{'Z': {'type': 'trimf', 'params': [0, 0.3, 0.6]},
 'PS': {'type': 'trimf', 'params': [0.3, 0.6, 0.9]},
 'PB': {'type': 'trimf', 'params': [0.6, 0.8, 1]}}

In [197]:
# We can also see the rules:
fis.rules

{0: 'inp1 is NB and inp2 is NB then out1 is Z',
 1: 'inp1 is Z and inp2 is Z then out1 is PS',
 2: 'inp1 is PB and inp2 is PB then out1 is PB'}

### example 2

Here we want to define a fuzzy system with only one input. As we will see, the only difference is in 
defining the rules

In [10]:
# first you should use the FuzzySets to define the first input:
inp_set1 = FuzzySets()

inp_set1.add_mf('NB', 'trimf', [-2, -1, 0]) 
inp_set1.add_mf('Z', 'trimf', [-1, 0, 1])
inp_set1.add_mf('PB', 'trimf', [0, 1, 2])




out_set1 = FuzzySets()
out_set1.rng = [0, 1] 

out_set1.add_mf('Z', 'trimf', [0, 0.3, 0.6])
out_set1.add_mf('PS', 'trimf', [0.3, 0.6, 0.9])
out_set1.add_mf('PB', 'trimf', [0.6, 0.8, 1])


fis = FIS(inputs={'inp1':inp_set1}, outputs = {'out1':out_set1})


# After defining the Fuzzy system, we should define the rules:

fis.add_rule('if inp1 is NB then out1 is Z')
fis.add_rule('if inp1 is Z then out1 is PS')
fis.add_rule('if inp1 is PB out1 is PB')

In [13]:
inputs = {'inp1': 1}
y = fis.run(inputs)
print(y)

{'out1': 0.8180032045008013}
